In [1]:
import pandas as pd
import numpy as np

import tensorflow as tf
from tensorflow import keras
from keras import layers, models
from keras.models import Sequential

from keras.preprocessing.text import Tokenizer
from keras import preprocessing

import matplotlib.pyplot as plt

In [2]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 

In [3]:
physical_devices = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)

In [4]:
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')

In [5]:
NUM_WORDS = 2000
MAX_LEN = 50
NUM_FEATURES = 6

In [6]:
tokenizer = Tokenizer(num_words=NUM_WORDS)
tokenizer.fit_on_texts(df_train['comment_text'])

sequences_train = tokenizer.texts_to_sequences(df_train['comment_text'])
sequences_test = tokenizer.texts_to_sequences(df_test['comment_text'])

In [7]:
x_train = preprocessing.sequence.pad_sequences(sequences_train, maxlen=MAX_LEN)
x_test = preprocessing.sequence.pad_sequences(sequences_test, maxlen=MAX_LEN)

ans_columns = df_train.columns[2:]

In [8]:
input_layer = keras.Input(shape=(None,))
x = layers.Embedding(NUM_WORDS + 1, 32, input_length=MAX_LEN)(input_layer)
x = layers.Bidirectional(layers.GRU(32, dropout=0.2))(x)

In [9]:
outputs_layers = list()
losses = list()
y_train = list()
for column in ans_columns:
    output_layer = layers.Dense(1, activation = 'sigmoid')(x)
    outputs_layers.append(output_layer)
    y_train.append(df_train[column].values)
    losses.append('binary_crossentropy')

In [10]:
model = keras.Model(inputs=[input_layer], outputs=outputs_layers)

In [11]:
model.compile(optimizer='rmsprop', loss=losses, metrics=['acc'])

In [17]:
x_train

array([[   0,    0,    0, ...,   73,   89,  985],
       [   0,    0,    0, ...,  992,  589,  182],
       [   0,    0,    0, ...,    1,  737,  468],
       ...,
       [   0,    0,    0, ...,  737,   23,   12],
       [   0,    0,    0, ...,  151,   34,   11],
       [   0,    0,    0, ...,    3, 1627,   88]], dtype=int32)

In [12]:
history = model.fit([x_train], y_train, epochs=25, batch_size=64)

Epoch 1/20
2494/2494 [==============================] - 19s 7ms/step - loss: 0.4474 - dense_loss: 0.1643 - dense_1_loss: 0.0330 - dense_2_loss: 0.0909 - dense_3_loss: 0.0195 - dense_4_loss: 0.1001 - dense_5_loss: 0.0396 - dense_acc: 0.9468 - dense_1_acc: 0.9897 - dense_2_acc: 0.9728 - dense_3_acc: 0.9966 - dense_4_acc: 0.9666 - dense_5_acc: 0.9905
Epoch 2/20
2494/2494 [==============================] - 17s 7ms/step - loss: 0.3572 - dense_loss: 0.1337 - dense_1_loss: 0.0253 - dense_2_loss: 0.0692 - dense_3_loss: 0.0146 - dense_4_loss: 0.0816 - dense_5_loss: 0.0328 - dense_acc: 0.9548 - dense_1_acc: 0.9903 - dense_2_acc: 0.9780 - dense_3_acc: 0.9970 - dense_4_acc: 0.9708 - dense_5_acc: 0.9912
Epoch 3/20
2494/2494 [==============================] - 17s 7ms/step - loss: 0.3467 - dense_loss: 0.1299 - dense_1_loss: 0.0247 - dense_2_loss: 0.0674 - dense_3_loss: 0.0137 - dense_4_loss: 0.0796 - dense_5_loss: 0.0314 - dense_acc: 0.9553 - dense_1_acc: 0.9905 - dense_2_acc: 0.9785 - dense_3_acc: 0

In [13]:
result = model.predict([x_test], 32)

In [14]:
result = np.array(result)
result = result.reshape(6, 153164)
result = result.T

In [15]:
df_ans = pd.DataFrame(result, columns=ans_columns)
df_ans.index = df_test['id']
df_ans

,toxic,severe_toxic,obscene,threat,insult,identity_hate
id,,,,,,
00001cee341fdb12,0.964374,0.155389,0.919356,8.093838e-03,0.854456,0.078526
0000247867823ef7,0.003424,0.000032,0.000836,8.670717e-07,0.000456,0.000035
00013b17ad220c46,0.002997,0.000095,0.000705,1.520311e-05,0.000477,0.000166
00017563c3f7919a,0.001874,0.000023,0.000611,4.940818e-06,0.000404,0.000071
00017695ad8997eb,0.033319,0.000455,0.007053,3.400265e-04,0.003800,0.000341
...,...,...,...,...,...,...
fffcd0960ee309b5,0.675277,0.002947,0.265462,2.438026e-04,0.085664,0.005211
fffd7a9a6eb32c16,0.028768,0.000283,0.006166,6.299827e-05,0.002732,0.000308
fffda9e8d6fafa9e,0.001542,0.000018,0.000567,1.651945e-06,0.000291,0.000034


In [16]:
df_ans.to_csv('ans.csv')